# pgvector quickstart


### Import libraries and environment variables

In [9]:
from __future__ import annotations

import dotenv


# Reload the variables in your '.env' file (override the existing variables)
dotenv.load_dotenv("../../.env", override=True)

True

In [ ]:
import os
import sys
from dotenv import load_dotenv
import logging



sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..'))) # Add the parent directory to the path since we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

from logging_utils import get_logger, global_log_config


global_log_config(
    log_level=logging.getLevelName("DEBUG"),
    json=False,
)


# # Load environment variables from a .env file
# load_dotenv()

# # Set the OpenAI API key environment variable
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

Logger set up with log level: 10


<loguru.logger handlers=[(id=2, level=10, sink=stdout)]>

2024-08-29 18:21:31.619 | DEBUG    | ipykernel.kernelbase:execute_request - kernelbase.py:802 | {'header': {'msg_id': '43763dae-7e976f631dffd87956d91e4a_15809_258', 'msg_type': 'execute_reply', 'username': 'malcolm', 'session': '43763dae-7e976f631dffd87956d91e4a', 'date': datetime.datetime(2024, 8, 29, 22, 21, 31, 618641, tzinfo=datetime.timezone.utc), 'version': '5.3'}, 'msg_id': '43763dae-7e976f631dffd87956d91e4a_15809_258', 'msg_type': 'execute_reply', 'parent_header': {'date': datetime.datetime(2024, 8, 29, 22, 21, 31, 586000, tzinfo=tzutc()), 'msg_id': '08c21b8b-04ce-4ac9-8caa-4977a6b210c0', 'msg_type': 'execute_request', 'session': '11abcf1d-6b84-4ea8-aed8-3562b139c4b3', 'username': '0d7d68c9-732a-4d67-8718-c95829b84087', 'version': '5.2'}, 'content': {'status': 'ok', 'execution_count': 10, 'user_expressions': {}, 'payload': []}, 'metadata': {'started': datetime.datetime(2024, 8, 29, 22, 21, 31, 589841, tzinfo=datetime.timezone.utc), 'dependencies_met': True, 'engine': 'cff99ef5-

### Read Docs

In [11]:
# SOURCE: https://www.youtube.com/watch?v=Ff3tJ4pJEa4
import pinecone
from dotenv import load_dotenv
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.vectorstores.pgvector import PGVector
from pgvector_service import PgvectorService
import os
import time

2024-08-29 18:21:36.453 | DEBUG    | ipykernel.kernelbase:dispatch_shell - kernelbase.py:419 | 
*** MESSAGE TYPE:execute_request*** | {}
2024-08-29 18:21:36.454 | DEBUG    | ipykernel.kernelbase:dispatch_shell - kernelbase.py:420 |    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': '# SOURCE: https://www.youtube.com/watch?v=Ff3tJ4pJEa4\nimport pinecone\nfrom dotenv import load_dotenv\nfrom langchain.document_loaders import TextLoader\nfrom langchain.embeddings.openai import OpenAIEmbeddings\nfrom langchain.text_splitter import CharacterTextSplitter\nfrom langchain.vectorstores import Pinecone\nfrom langchain.document_loaders import TextLoader\nfrom langchain.vectorstores.pgvector import PGVector\nfrom pgvector_service import PgvectorService\nimport os\nimport time'}
   --->
    | {}
2024-08-29 18:21:36.454 | DEBUG    | ipykernel.kernelbase:dispatch_shell - kernelbase.py:429 | execute_request: {'header': {'da

# Load the documents

> 4. Initialize a LangChain vector store
> Now that you've built your Pinecone index, you need to initialize a LangChain vector store using the index. This step uses the OpenAI API key you set as an environment variable earlier. Note that OpenAI is a paid service and so running the 
> remainder of this tutorial may incur some small cost.

In [12]:
loader = TextLoader(
    "../../data/The Project Gutenberg eBook of A Christmas Carol in Prose; Being a Ghost Story of Christmas.txt"
)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()

from langchain_openai import OpenAIEmbeddings
# get openai api key from platform.openai.com
model_name = 'text-embedding-ada-002'
embeddings = OpenAIEmbeddings(
    model=model_name,
)

query = "The Project Gutenberg eBook of A Christmas Carol in Prose; Being a Ghost Story of Christmas"


"""
First, we compare to Pinecone, a managed vector store service.

"""

'\nFirst, we compare to Pinecone, a managed vector store service.\n\n'

# Create / Load the Pinecone index

In [27]:
use_serverless = True

# pinecone.init(
#     api_key=os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENV")
# )

# SOURCE: https://docs.pinecone.io/integrations/langchain#4-initialize-a-langchain-vector-store

import os
# from pinecone.control.pinecone import Pinecone
from pinecone.grpc.pinecone import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec, PodSpec
import time
from langchain_pinecone import PineconeVectorStore
import rich

pc: Pinecone = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

if use_serverless:
    spec = ServerlessSpec(cloud='aws', region='us-east-1')
else:
    # if not using a starter index, you should specify a pod_type too
    spec = PodSpec()

index_name = "demo-index"


# # delete all indexes
# if index_name in pc.list_indexes().names():0
#     pc.delete_index(index_name)

# for pinecone_index in pc.list_indexes():
rich.print(pc.list_indexes())
rich.print(pc.list_indexes().names())

# for pinecone_index in pc.list_indexes():
#     print(pinecone_index)
#     if index_name not in pinecone_index.names():
#         pc.create_index(name=index_name, metric="cosine", dimension=1536, spec=spec)

#         # wait for index to be initialized
#         while not pc.describe_index(index_name).status['ready']:
#             time.sleep(1)

#         vectorstore = PineconeVectorStore(
#             index_name=index_name, embedding=embeddings, text_key="text"
#         )
#         vectorstore.add_documents(docs)

#     else:
#         print(f"Index {index_name} already exists")
#         vectorstore = PineconeVectorStore(
#             index_name=index_name, embedding=embeddings, text_key="text"
#         )
if index_name not in pc.list_indexes().names():

    pc.create_index(name=index_name, metric="cosine", dimension=1536, spec=spec)

    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

    vectorstore = PineconeVectorStore(
        index_name=index_name, embedding=embeddings, text_key="text"
    )
    vectorstore.add_documents(docs)

else:
    print(f"Index {index_name} already exists")
    vectorstore = PineconeVectorStore(
        index_name=index_name, embedding=embeddings, text_key="text"
    )

    # pinecone_docsearch = pc.from_documents(
    #     docs, embeddings, index_name="demo-index"
    # )
# else:
#     pinecone_docsearch = pc.from_existing_index(index_name, embeddings)

# rich.inspect(pc, all=True)

index = pc.Index(index_name)
index.describe_index_stats()

{'indexes': []}

[]

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

# Initialize the LangChain vector store: 

> https://docs.pinecone.io/integrations/langchain#4-initialize-a-langchain-vector-store


The text_field parameter sets the name of the metadata field that stores the raw text when you upsert records using a LangChain operation such as vectorstore.from_documents or vectorstore.add_texts. This metadata field is used as the page_content in the Document objects retrieved from query-like LangChain operations such as vectorstore.similarity_search. If you do not specify a value for text_field, it will default to "text".

In [18]:
from langchain_pinecone import PineconeVectorStore
text_field = "text"
vectorstore = PineconeVectorStore(
    index_name=index_name, embedding=embeddings, text_key="text"
)

# Now you can query the vector store directly using vectorstore.similarity_search:

In [17]:
vectorstore.similarity_search(
    query,  # our search query
    k=4  # return 3 most relevant docs
)

[]

# Query the index with LanChain

In [ ]:
def run_query_pinecone(docsearch: PineconeVectorStore, query: str):
    docs = docsearch.similarity_search(query, k=4)
    result = docs[0].page_content
    return result


def calculate_average_execution_time(func, *args, **kwargs):
    total_execution_time = 0
    num_runs = 10
    for _ in range(num_runs):
        start_time = time.time()
        result = func(*args, **kwargs)  # Execute the function with its arguments
        end_time = time.time()
        execution_time = end_time - start_time
        total_execution_time += execution_time
    average_execution_time = round(total_execution_time / num_runs, 2)
    print(result)
    print(
        f"\nThe function took an average of {average_execution_time} seconds to execute."
    )
    return


calculate_average_execution_time(
    run_query_pinecone, docsearch=vectorstore, query=query
)


"""
Now, we compare to PGVector, an open source vector store service.

"""

# Create a PGVector Store

In [ ]:

"""
Donwload postgresql to run locally:
https://www.postgresql.org/download/

How to install the pgvector extension:
https://github.com/pgvector/pgvector

Fix common installation issues:
https://github.com/pgvector/pgvector?tab=readme-ov-file#installation-notes
"""

COLLECTION_NAME = "The Project Gutenberg eBook of A Christmas Carol in Prose"

CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg3"),
    host=os.environ.get("PGVECTOR_HOST", "localhost"),
    port=int(os.environ.get("PGVECTOR_PORT", "6432")),
    database=os.environ.get("PGVECTOR_DATABASE", "langchain"),
    user=os.environ.get("PGVECTOR_USER", "langchain"),
    password=os.environ.get("PGVECTOR_PASSWORD", "langchain"),
)

# create the store
db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=False,
)

# load the store
pgvector_docsearch = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
)


# Query the index with PGVector

In [ ]:

def run_query_pgvector(docsearch, query):
    docs = docsearch.similarity_search(query, k=4)
    result = docs[0].page_content
    return result


calculate_average_execution_time(
    run_query_pgvector, docsearch=pgvector_docsearch, query=query
)


# Add more collections to the database

In [ ]:

loader = TextLoader("../../data/The Project Gutenberg eBook of Romeo and Juliet.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
new_docs = text_splitter.split_documents(documents)


COLLECTION_NAME_2 = "The Project Gutenberg eBook of Romeo and Juliet"

db = PGVector.from_documents(
    embedding=embeddings,
    documents=new_docs,
    collection_name=COLLECTION_NAME_2,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=False,
)

# Query the index with multiple collections


In [ ]:

pg = PgvectorService(CONNECTION_STRING)


def run_query_multi_pgvector(docsearch, query):
    docs = docsearch.custom_similarity_search_with_scores(query, k=4)
    result = docs[0][0].page_content
    print(result)


run_query_multi_pgvector(pg, query)

# --------------------------------------------------------------
# Delete the collection
# --------------------------------------------------------------
pg.delete_collection(COLLECTION_NAME)
pg.delete_collection(COLLECTION_NAME_2)

# --------------------------------------------------------------
# Update the collection
# --------------------------------------------------------------
pg.update_collection(docs=docs, collection_name=COLLECTION_NAME)
